In [1]:
import pygame, time
import sys, os, getopt, math, random
import numpy as np
from constants import *

pygame 2.1.2 (SDL 2.0.18, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
swarm_size = 3  # default size of the swarm
manual_mode = False  # manually press enter key to proceed between simulations

In [3]:
def S14_closest_robot(robot_host, robot_neighbors):
    # "robot_host": the robot to measure distance from
    # "robot_neighbors": a list of robots to be compared with
    robot_closest = robot_neighbors[0]
    dist_closest = dist_table[robot_host,robot_closest]
    for i in robot_neighbors[1:]:
        dist_temp = dist_table[robot_host,i]
        if dist_temp < dist_closest:
            robot_closest = i
            dist_closest = dist_temp
    return robot_closest

In [130]:
def check_amoeba(poses_t, poses, num_robots, step, thresh = 1):
    """
    Function to check if a given amoeboid swarm is amoeboid after movement of the swarm robots

    Args:
        poses_t (np.array): Original positions of the robots
        poses (np.array): New positions of teh robors
        num_robots (int): Number of robots currently in the swarm
        step (float): step moving distance
        thresh (float, optional): threshold value for centroid and distance from centroid conditions. Defaults to 1.

    Returns:
        bool: True if the new swarm is amoeboid, False otherwise
    """    
    centroid = getcenteroid(poses[:num_robots])
    centroid_t = getcenteroid(poses_t[:num_robots])
    # print(centroid)
    arr_radii = np.array([get_dist(centroid, p) for p in poses[:num_robots]])
    arr_radii_t = np.array([get_dist(centroid, p) for p in poses_t[:num_robots]])
    
    delta_centroid = get_dist(centroid, centroid_t)
    delta_radius = np.mean(arr_radii) - np.mean(arr_radii_t)
    
    if delta_centroid/step < thresh and delta_radius/np.mean(arr_radii_t) < thresh:
        return True
    return False
    
    

# Function to get centroid
def getcenteroid(arr):
    length, dim = arr.shape
    return np.array([np.sum(arr[:, i])/length for i in range(dim)])

def get_dist(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def euclidean_distance(a, b):
    """Euclidean Distance

    Args:
        a (np.array): a
        b (np.array): b

    Returns:
        float: dist
    """
    return np.linalg.norm(a-b)


In [5]:
# general function to steer robot away from wall if out of boundary (following physics)
# use global variable "world_side_length"
def robot_boundary_check(robot_pos, robot_ori):
    new_ori = robot_ori
    if robot_pos[0] >= world_side_length:  # outside of right boundary
        if math.cos(new_ori) > 0:
            new_ori = reset_radian(2*(math.pi/2) - new_ori)
            # further check if new angle is too much perpendicular
            if new_ori > 0:
                if (math.pi - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
            else:
                if (new_ori + math.pi) < perp_thres:
                    new_ori = new_ori + devia_angle
    elif robot_pos[0] <= 0:  # outside of left boundary
        if math.cos(new_ori) < 0:
            new_ori = reset_radian(2*(math.pi/2) - new_ori)
            if new_ori > 0:
                if new_ori < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (-new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    if robot_pos[1] >= world_side_length:  # outside of top boundary
        if math.sin(new_ori) > 0:
            new_ori = reset_radian(2*(0) - new_ori)
            if new_ori > -math.pi/2:
                if (new_ori + math.pi/2) < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (-math.pi/2 - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    elif robot_pos[1] <= 0:  # outside of bottom boundary
        if math.sin(new_ori) < 0:
            new_ori = reset_radian(2*(0) - new_ori)
            if new_ori > math.pi/2:
                if (new_ori - math.pi/2) < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (math.pi/2 - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    return new_ori

# # general function to steer robot away from wall if out of boundary (in random direction)
# # use global variable "world_side_length"
# def robot_boundary_check(robot_pos, robot_ori):
#     new_ori = robot_ori
#     if robot_pos[0] >= world_side_length:  # outside of right boundary
#         if math.cos(new_ori) > 0:
#             new_ori = reset_radian(math.pi/2 + np.random.uniform(0,math.pi))
#     elif robot_pos[0] <= 0:  # outside of left boundary
#         if math.cos(new_ori) < 0:
#             new_ori = reset_radian(-math.pi/2 + np.random.uniform(0,math.pi))
#     if robot_pos[1] >= world_side_length:  # outside of top boundary
#         if math.sin(new_ori) > 0:
#             new_ori = reset_radian(-math.pi + np.random.uniform(0,math.pi))
#     elif robot_pos[1] <= 0:  # outside of bottom boundary
#         if math.sin(new_ori) < 0:
#             new_ori = reset_radian(0 + np.random.uniform(0,math.pi))
#     return new_ori

# general function to reset radian angle to [-pi, pi)
def reset_radian(radian):
    while radian >= math.pi:
        radian = radian - 2*math.pi
    while radian < -math.pi:
        radian = radian + 2*math.pi
    return radian

In [6]:
# calculate world_side_coef from a desired screen size for 30 robots
def cal_world_side_coef():
    desired_screen_size = 500  # desired screen size for 30 robots
    desired_world_size = float(desired_screen_size) / pixels_per_length
    return desired_world_size / pow(30, 1/power_exponent)
world_side_coef = cal_world_side_coef()
# world_side_length = world_side_coef * pow(swarm_size, 1/power_exponent)
world_side_length = world_side_coef * pow(30, 1/power_exponent)
world_size = (world_side_length, world_side_length)  # square physical world
# screen size calculated from world size
screen_side_length = int(pixels_per_length * world_side_length)
screen_size = (screen_side_length, screen_side_length)  # square display world

In [7]:
# Function to create a unit vector from a given vector
def unit_vector(vector):
    return np.abs(vector / np.linalg.norm(vector))

In [8]:
try:
    pygame.display.quit()
    pygame.quit()
except:
    pass
pygame.init()

font = pygame.font.SysFont("Cabin", 12)

screen = pygame.display.set_mode(screen_size)
pygame.display.set_caption("Demo 1")
# draw the network
screen.fill(color_white)

pygame.display.update()


In [9]:
def convert_coordinates(pos):
    poses_temp = pos / world_side_length
    poses_temp[1] = 1.0 - poses_temp[1]
    poses_temp = poses_temp * screen_side_length
    disp = poses_temp.astype(int) 
    return disp

In [167]:
screen.fill(color_white)

# robot properties
# robot_poses = np.random.rand(swarm_size, 2) * world_side_length  # initialize the robot poses
robot_poses = np.array([[1,1] for i in range(swarm_size)])  # initialize the robot poses
dist_table = np.zeros((swarm_size, swarm_size))  # distances between robots
conn_table = np.zeros((swarm_size, swarm_size))  # connections between robots
    # 0 for disconnected, 1 for connected
conn_lists = [[] for i in range(swarm_size)]  # lists of robots connected
# function for all simulations, update the distances and connections between the robots
def dist_conn_update():
    global dist_table
    global conn_table
    global conn_lists
    conn_lists = [[] for i in range(swarm_size)]  # empty the lists
    for i in range(swarm_size):
        for j in range(i+1, swarm_size):
            dist_temp = np.linalg.norm(robot_poses[i] - robot_poses[j])
            dist_table[i,j] = dist_temp
            dist_table[j,i] = dist_temp
            if dist_temp > comm_range:
                conn_table[i,j] = 0
                conn_table[j,i] = 0
            else:
                conn_table[i,j] = 1
                conn_table[j,i] = 1
                conn_lists[i].append(j)
                conn_lists[j].append(i)
dist_conn_update()  # update the distances and connections
disp_poses = []  # display positions
# function for all simulations, update the display positions
def disp_poses_update():
    global disp_poses
    poses_temp = robot_poses / world_side_length
    poses_temp[:,1] = 1.0 - poses_temp[:,1]
    poses_temp = poses_temp * screen_side_length
    disp_poses = poses_temp.astype(int)  # convert to int and assign to disp_poses
    # disp_poses = np.array([[0,0] for i in range(swarm_size)])  # 
disp_poses_update()
# deciding the seed robots, used in simulations with moving robots
seed_percentage = 0.1  # the percentage of seed robots in the swarm
seed_quantity = min(max(int(swarm_size*seed_percentage), 1), swarm_size)
    # no smaller than 1, and no larger than swarm_size
robot_seeds = [False for i in range(swarm_size)]  # whether a robot is a seed robot
    # only seed robot can initialize the forming a new group
seed_list_temp = np.arange(swarm_size)
np.random.shuffle(seed_list_temp)
for i in seed_list_temp[:seed_quantity]:
    robot_seeds[i] = True
    
current_swarm_size = 1  # the current swarm size
sim_haulted = False
# set up the simulation window

disp_poses_update()

robot_oris = np.random.rand(swarm_size) * 2 * math.pi - math.pi  # in range of [-pi, pi)
# use step moving distance in each update, instead of calculating from robot velocity
# so to make it independent of simulation frequency control
step_moving_dist = 0.05  # should be smaller than destination distance error
destination_error = 0.1
mov_vec_ratio = 0.5  # ratio used when calculating mov vector
# spring constants in SMA
linear_const = 1.0
bend_const = 0.8
disp_coef = 0.5
# for avoiding space too small on loop
space_good_thres = desired_space * 0.85


for i in range(swarm_size):
    pygame.draw.circle(screen, color_black, disp_poses[i], robot_size_formation,
        robot_width_empty)
    # pygame.draw.circle(screen, color_black, disp_poses[i], int(comm_range*pixels_per_length), 1)
pygame.display.update()

# robot_poses, current_swarm_size

In [187]:
step =1
init = 100
for iter in range(1):
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:  # close window button is clicked
            print("program exit in simulation 1")
            sys.exit()  # exit the entire program
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_SPACE:
                sim_haulted = not sim_haulted  # reverse the pause flag
    # if sim_haulted: continue
    print(f"Halted = {sim_haulted}, swarm size = {current_swarm_size}")
    print(f"current swarm size is: {current_swarm_size} out of {swarm_size}")
    current_pos = robot_poses[0]
    
    robot_poses_t = robot_poses.copy()
    if current_swarm_size == 0: pass #continue
    elif current_swarm_size < 3:
        # random_vector = unit_vector((np.random.rand(2)-0.5)*2)
        for j in range(current_swarm_size):
            point = np.random.randint(0, 100, size=2)
            print(point)
            robot_oris[j] = math.atan2(point[0], point[1])
            # if current_swarm_size < swarm_size:
            #     robot_oris[j] = np.absolute(robot_oris[j])
        
            robot_oris[j] = robot_boundary_check(robot_poses_t[j], robot_oris[j])
            robot_poses[j] = robot_poses_t[j] + (step * np.array([math.cos(robot_oris[j]), math.sin(robot_oris[j])]))
    else:
        print("start")
        centroid = (np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1]))
        # Pseudopod Grouping
        init = np.random.randint(0, current_swarm_size)
        print("Robot chosen is: ", init)
        # print(dist_table[init, :])
        robot_ids = list(range(current_swarm_size))
        robot_ids.remove(init)
        for div in range(current_swarm_size//3):
            closest = S14_closest_robot(init, robot_ids)
            print("closest robot id is: ", closest)
            # Choose another robot that is closest to the two
            print(robot_ids)
            robot_ids.remove(closest)
            print(robot_ids)
            closest1 = S14_closest_robot(init, robot_ids)
            closest2 = S14_closest_robot(closest, robot_ids)
            if closest1 == closest2:
                triad = [init, closest, closest1]
            else:
                if dist_table[init, closest1] < dist_table[closest, closest2]:
                    triad = [init, closest, closest1]
                else:
                    triad = [init, closest, closest2]
            print("triad is: ", triad) # Direction of Last robot in this list is updated, other two are pseudopods
            dist = [euclidean_distance(robot_poses[triad[i]], centroid) for i in triad]
            
            dist.sort(reverse=True)
            vector1 = np.array([math.cos(robot_oris[triad[0]]), math.sin(robot_oris[triad[0]])])
            vector2 = np.array([math.cos(robot_oris[triad[1]]), math.sin(robot_oris[triad[1]])])
            point = np.random.randint(-100, 100, size=2)
            # print(point)
            # robot_oris[j] = math.atan2(point[0], point[1])
            vector3 = unit_vector(point)
            new = vector1 + vector2 + np.array(vector3)
            print(new)
            robot_oris[triad[2]] = math.atan2(new[1], new[0])
            robot_poses[triad[2]] = robot_poses_t[triad[2]] + (step * np.array([math.cos(robot_oris[triad[2]]), math.sin(robot_oris[triad[2]])]))
            robot_oris[triad[2]] = robot_boundary_check(robot_poses_t[triad[2]], robot_oris[triad[2]])
            # np.delete(all_roboots, closest)
            # closest2 = S14_closest_robot(init, all_robots)
            # print(closest, closest2)
        # for i in sorted(list(range(current_swarm_size)), key=lambda k: random.random()): 
            # while True:
            #     point = np.absolute(np.random.randint(-100, 100, size=2))
            #     robot_oris[i] = math.atan2(point[0], point[1])
            #     new = robot_poses.copy()
            #     robot_oris[i] = robot_boundary_check(robot_poses_t[i], robot_oris[i])
            #     new[i] = robot_poses_t[i] + (step * np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])]))
            #     print("vector ",np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])]))
            #     print("1 ", robot_poses_t[i] + (1 * np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])])))
            #     print("2, ", robot_poses_t[i] + (1.5 * np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])])))
            #     if check_amoeba(robot_poses_t, new, current_swarm_size+1, step, thresh=1):
            #         robot_poses = new
            #         break 
            # pass
                

            
    print("robot positions: ", robot_poses[:current_swarm_size]) 
    
    if current_swarm_size < swarm_size:
        current_swarm_size+=1 
    dist_conn_update()
    disp_poses_update()
    screen.fill(color_white)
    # disp_poses=robot_poses
    for i in range(swarm_size):
        if i != init:
            pygame.draw.circle(screen, distinct_color_set[i], disp_poses[i], robot_size_formation, robot_width_empty)
        else:
            pygame.draw.circle(screen, color_black, disp_poses[i], robot_size_formation, robot_width_empty)
        pygame.draw.line(screen, color_grey, disp_poses[i], convert_coordinates(robot_poses[i]+np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])])*0.5))
        # pygame.draw.line(screen, color_grey, disp_poses[i], robot_poses[i]+np.array([math.cos(robot_oris[i]), math.sin(robot_oris[i])])*0.5)
        
        # pygame.draw.circle(screen, color_black, disp_poses[i], int(comm_range*pixels_per_length), 1)
        
    # Plot centroid
    pygame.draw.circle(screen, color_red, convert_coordinates(getcenteroid(robot_poses[:current_swarm_size])), 4)
    pygame.display.update()
    # time.sleep(0.1)

Halted = False, swarm size = 3
current swarm size is: 3 out of 3
start
Robot chosen is:  0
closest robot id is:  2
[1, 2]
[1]
triad is:  [0, 2, 1]
[2.40208486 1.71849302]
robot positions:  [[1 1]
 [2 1]
 [1 1]]


In [218]:
np.absolute(robot_oris[j])

2.4186133001883023

In [63]:
robot_poses

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [64]:
random_vector = unit_vector((np.random.rand(2)-0.5)*2)
robot_poses[0] = current_pos + random_vector

In [148]:
math.atan2(-1, -1)

-2.356194490192345

In [227]:
10//3

3

In [41]:
np.array([1,2])+np.array([3,5])

array([4, 7])